In [1]:
import pypyodbc
import sys
import pandas as pd 
import time
# sys.path.append('../')
from KNNmodel import *
from rec_helper import *
from tqdm import tqdm
import scipy.sparse as sp          

In [2]:
con = pypyodbc.connect("DRIVER={SQL Server};SERVER=dbm_public;UID=sa;PWD=01060728;DATABASE=project2017")

In [3]:
df = pd.read_sql("select * from 基金推薦_近二年申購_憑證歸戶 ",con)

In [4]:
df.head()

,身分證字號,憑證,基金代碼,商品投資屬性,申購登錄日,扣款次數,申購扣款金額_台幣,國內外基金註記,aum基金型態別,投資型態,投資地區,型態別,aum型態別,aum計算類別
0,G2723186890,120J0W41600368,J0W,RR3,20160701,1,234180.8,1.0,B,b.單筆申購,a.境外基金,b.債券型,b.債券型,國外債券型
1,A1235025000,136317C1700029,317,RR3,20170811,1,496793.0,0.0,B,b.單筆申購,b.國內基金,b.債券型,b.債券型,國內債券型
2,F2778691990,18463Z51700045,63Z,RR5,20171011,3,13571.1,1.0,E,a.定時定額,a.境外基金,a.股票型,a.股票型,國外股票型
3,D2724924750,023Y3801700178,Y38,RR4,20171027,1,500000.0,1.0,E,b.單筆申購,a.境外基金,a.股票型,a.股票型,國外股票型
4,D2721254300,013J1F11600058,J1F,RR3,20160504,4,2000000.0,1.0,W,a.定時定額,a.境外基金,d.其他型,d.平衡型,國外其他型


In [5]:
# 建立u-i 矩陣 至少買過一檔基金
df_gt2 = threshold_interaction(df,rowname='身分證字號',colname='基金代碼',row_min=1,col_min=0) 

Starting interactions info
Number of rows: 35122
Number of cols: 2059
Sparsity: 0.20%
Ending interactions info
Number of rows: 19748
Number of columns: 2028
Sparsity: 0.32%


In [7]:
purchased_ui, userid_to_idx, \
    idx_to_userid, itemid_to_idx,idx_to_itemid  = df_to_spmatrix(df_gt2,'身分證字號','基金代碼',binary=False)
train,test, user_idxs = train_test_split(purchased_ui,split_count=1,fraction=0.2)     

_____

## KNN model 
ubcf/ibcf/popular 
### jaccard similarity

In [8]:
## ibcf
t1 = time.time()
model_i = KNNmodel(train,kind='ibcf')
model_i.jaccard_sim()
model_i.fit(topK=100,remove=True)
t2 = time.time()
dt_ibcf = t2-t1
print('\n***')
print('time cost for ibcf:{:.1f}s'.format(dt_ibcf))

similarity (jaccard) matrix built (ibcf), 
sparsity of similarity: 8.20 %


100%|███████████████████████████████████| 2028/2028 [00:32<00:00, 61.95items/s]


ibcf rating matrix built...

handling nan data...


100%|█████████████████████████| 12315828/12315828 [00:09<00:00, 1244660.52it/s]


numbers of rows : 19748
numbers of cols: 2028
sparsity of rating: 30.75 %
save into *.rating attribute...

***
time cost for ibcf:53.8s


In [9]:
## ubcf -- jaccard
t1 = time.time()
model_u = KNNmodel(train,kind='ubcf')
model_u.jaccard_sim()
model_u.fit(topK=100,remove=True)
t2 = time.time()
dt_ubcf = t2-t1
print('\n***')
print('time cost for ubcf:{:.1f}s'.format(dt_ubcf))

similarity (jaccard) matrix built (ubcf), 
sparsity of similarity: 11.19 %
-- start building topK user array...


100%|███████████████████████████████| 19748/19748 [00:16<00:00, 1232.06users/s]


-- end building topK user array---
start building prediction rating...


100%|████████████████████████████████| 19748/19748 [00:32<00:00, 616.95users/s]


ubcf rating matrix built...

handling nan data...


100%|███████████████████████████| 2230362/2230362 [00:01<00:00, 1222353.75it/s]


numbers of rows : 19748
numbers of cols: 2028
sparsity of rating: 5.57 %
save into *.rating attribute...

***
time cost for ubcf:72.9s


In [14]:
## popular 
t1 = time.time()
model_p = KNNmodel(train,kind='popular')
model_p.fit(topK=100,remove=True)
t2 = time.time()
dt_pop = t2-t1
print('\n***')
print('time cost for popular:{:.1f}s'.format(dt_pop))

100%|██████████████████████████████████| 19748/19748 [00:03<00:00, 6331.89it/s]


popular rating matrix built...

handling nan data...


100%|███████████████████████████| 1974800/1974800 [00:01<00:00, 1194319.68it/s]


numbers of rows : 19748
numbers of cols: 535
sparsity of rating: 18.69 %
save into *.rating attribute...

***
time cost for popular:5.7s


* UBCF / IBCF / popular
## cosine similarity

In [17]:
t1 = time.time()
model_i_cos = KNNmodel(train,kind='ibcf')
model_i_cos.cosine_sim()
model_i_cos.fit(topK=100,remove=True)
t2 = time.time()
dt_ibcf = t2-t1
print('\n***')
print('time cost for ibcf:{:.1f}s,(cosine)'.format(dt_ibcf))

C:\Program Files\Anaconda3\envs\py36\lib\site-packages\scipy\sparse\compressed.py:774: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


similarity (cosine) matrix build (ibcf), 
sparsity of similarity: 8.25 %


100%|███████████████████████████████████| 2028/2028 [00:35<00:00, 56.65items/s]


ibcf rating matrix built...

handling nan data...


100%|█████████████████████████| 14208785/14208785 [00:10<00:00, 1292773.67it/s]


numbers of rows : 19748
numbers of cols: 2028
sparsity of rating: 35.48 %
save into *.rating attribute...

***
time cost for ibcf:60.2s,(cosine)


In [10]:
## ubcf
t1 = time.time()
model_u_cos = KNNmodel(train,kind='ubcf')
model_u_cos.cosine_sim()
model_u_cos.fit(topK=100,remove=True)
t2 = time.time()
dt_ubcf = t2-t1
print('\n***')
print('time cost for ubcf:{:.1f}s'.format(dt_ubcf))

similarity (cosine) matrix build (ubcf), 
sparsity of similarity: 11.20 %
-- start building topK user array...


100%|███████████████████████████████| 19748/19748 [00:17<00:00, 1145.11users/s]


-- end building topK user array---
start building prediction rating...


100%|████████████████████████████████| 19748/19748 [00:32<00:00, 611.33users/s]


ubcf rating matrix built...

handling nan data...


100%|███████████████████████████| 2513476/2513476 [00:02<00:00, 1174008.84it/s]


numbers of rows : 19748
numbers of cols: 2028
sparsity of rating: 6.28 %
save into *.rating attribute...

***
time cost for ubcf:54.9s


* bm25 

In [11]:
t1 = time.time()
model_i_bm25 = KNNmodel(train,kind='ibcf')
model_i_bm25.bm25_sim()
model_i_bm25.fit(topK=100,remove=True)
t2 = time.time()
dt_ibcf = t2-t1
print('\n***')
print('time cost for ibcf:{:.1f}s'.format(dt_ibcf))

100%|███████████████████████████████████| 2028/2028 [00:32<00:00, 62.27items/s]


ibcf rating matrix built...

handling nan data...


100%|█████████████████████████| 11350905/11350905 [00:08<00:00, 1320524.15it/s]


numbers of rows : 19748
numbers of cols: 2028
sparsity of rating: 28.34 %
save into *.rating attribute...

***
time cost for ibcf:51.4s


In [12]:
t1 = time.time()
model_u_bm25 = KNNmodel(train,kind='ubcf')
model_u_bm25.bm25_sim(K1=10,B=1)
model_u_bm25.fit(topK=100,remove=True)
t2 = time.time()
dt_ubcf = t2-t1
print('\n***')
print('time cost for ubcf:{:.1f}s'.format(dt_ubcf))

-- start building topK user array...


100%|███████████████████████████████| 19748/19748 [00:17<00:00, 1141.20users/s]


-- end building topK user array---
start building prediction rating...


100%|████████████████████████████████| 19748/19748 [00:31<00:00, 622.29users/s]


ubcf rating matrix built...

handling nan data...


100%|███████████████████████████| 2011461/2011461 [00:01<00:00, 1212820.32it/s]


numbers of rows : 19748
numbers of cols: 2028
sparsity of rating: 5.02 %
save into *.rating attribute...

***
time cost for ubcf:53.7s


## 評估模型 
    - recall/ precision


In [15]:
uids = np.arange(0,train.shape[0])
print('======jaccard similarity =====')
predall_u = model_u.predict(uids,topN=10) # np array (itemidx)
model_u.evaluate(predall_u,test,method='recall') # 29.27

predall_i = model_i.predict(uids,topN=10) #nparray (itemidx)
model_i.evaluate(predall_i,test,method='recall') # 12.71 

predall_p = model_p.predict(uids,topN=10) #nparray (itemidx)
model_p.evaluate(predall_p,test,method='recall') # 20.08

======jaccard similarity =====

-------------
model: ubcf,
topN: 10
recall:27.91 %

-------------
model: ibcf,
topN: 10
recall:13.67 %

-------------
model: popular,
topN: 10
recall:20.99 %


In [18]:
print('=====cosine similarity======')
predall_u_cos = model_u_cos.predict(uids,topN=10)
model_u_cos.evaluate(predall_u_cos,test,method='recall')

predall_i_cos = model_i_cos.predict(uids,topN=10)
model_i_cos.evaluate(predall_i_cos,test,method='recall')

=====cosine similarity======

-------------
model: ubcf,
topN: 10
recall:27.42 %

-------------
model: ibcf,
topN: 10
recall:1.54 %


In [19]:
uids = np.arange(0,train.shape[0])
print('======bm25 similarity =====')
predall_u_bm25 = model_u_bm25.predict(uids,topN=10) # np array (itemidx)
model_u_bm25.evaluate(predall_u_bm25,test,method='recall') # 29.27

predall_i_bm25 = model_i_bm25.predict(uids,topN=10) #nparray (itemidx)
model_i_bm25.evaluate(predall_i_bm25,test,method='recall')  

# predall_p = model_p.predict(uids,topN=10) #nparray (itemidx)
# model_p.evaluate(predall_p,test,method='recall') # 20.08

======bm25 similarity =====

-------------
model: ubcf,
topN: 10
recall:27.48 %

-------------
model: ibcf,
topN: 10
recall:5.27 %


## 推薦清單 

In [39]:
fundid_names_df = pd.read_csv('../funds/fundid_to_name.csv',encoding='cp950')
fundid_to_names = {}

for d in fundid_names_df.to_dict('records'):
    fundid_to_names[d['基金代碼']] = d['基金中文名稱']
#%% 

In [68]:
def sample_recommendation_knn(data,model,user_idxs, print_output=True):
    ## user_ids : list of user ids ,eg: ['A1234567890','B12345xxxxxx',...]
    train = data['train']
    test = data['test']
    idx_to_itemid = data['idx_to_itemid']
    idx_to_userid = data['idx_to_userid']
    
    uidxs = np.array(user_idxs) # convert to uidxs np.array
    rec_fundidxs = model.predict(uidxs)
    for _, uidx in enumerate(user_idxs):
        known_positives_itemids = [ 
            idx_to_itemid[e] for e in train[uidx].indices
        ]
        known_positives_item_names = [
            fundid_to_names[fundid] for fundid in known_positives_itemids
        ]
        
        if print_output == True:
            print("User %s" % idx_to_userid[uidx])
            print("     ===Known positives(top3)===")

            for x in known_positives_item_names[:3]:
                print("        %s" % x)

            print("     ===Recommended(top3)===")

            for x in rec_fundidxs[_,:3]:
                print("        %s" % fundid_to_names[idx_to_itemid[x]])
            print('\n')

In [69]:
data = {'train': train, 'test':test, 'idx_to_itemid': idx_to_itemid, 'idx_to_userid':idx_to_userid}
sample_recommendation_knn(data,model_u,[4,10,20])

User D2721254300
     ===Known positives(top3)===
        (百元基金)摩根策略總報酬基金-JPM-A股累計(美元對沖)
        (百元基金)永豐滬深300紅利指數基金
        鋒裕新興市場債券基金TXD配息(美元)
     ===Recommended(top3)===
        路博邁投資基金-NB新興市場股票基金T累積類股(美元)
        (百元基金)貝萊德世界礦業基金(美元)
        (百元基金)摩根俄羅斯基金-JPM-A股分派(美元)


User N2221823430
     ===Known positives(top3)===
        (百元基金)摩根策略總報酬基金-JPM-A股累計(美元對沖)
        摩根亞太入息基金-JPM-A股每月派息(美元)
        摩根多重收益基金-JPM-A股每月派息(美元對沖)
     ===Recommended(top3)===
        安聯收益成長基金-AM(穩定月收類股)(美元)
        路博邁投資基金-NB新興市場股票基金T累積類股(美元)
        路博邁AR台灣股票基金-T月配(新臺幣)


User D2297379550
     ===Known positives(top3)===
        聯博全球高收益債券基金TA月配(人民幣)
        (百元基金)摩根策略總報酬基金-JPM-A股累計(美元對沖)
        (百元基金)摩根俄羅斯基金-JPM-A股分派(美元)
     ===Recommended(top3)===
        (百元基金)富蘭克林坦伯頓全球生技領航基金(美元)
        (百元基金)貝萊德世界黃金基金(美元)
        安聯收益成長基金-AM(穩定月收類股)(南非幣避險)




## implicit ALS

In [7]:
import implicit

In [8]:
# initialize a model
model_als = implicit.als.AlternatingLeastSquares(factors=50)

train_64 = train.astype('float64')
# train the model on a sparse matrix of item/user/confidence weights
model_als.fit(train_64.T) 

# recommend items for a user
# user_items = item_user_data.T.tocsr()
useridx = 10
recommendations = model_als.recommend(0, train_64)

# find related items
itemidx = 10 
related = model_als.similar_items(itemidx)

In [9]:
print('shape of user_factor:',model_als.user_factors.shape)
print('shape of item_factor:',model_als.item_factors.shape)


shape of user_factor: (19748, 50)
shape of item_factor: (2028, 50)


In [11]:
from sklearn.metrics import mean_squared_error
def calculate_mse(model, ratings, user_index=None):
    """Recommend products for all customers"""
    preds = model.user_factors.dot(model.item_factors.T)
#     preds = model.predict_for_customers()
    if user_index:
        return mean_squared_error(ratings[user_index, :].toarray().ravel(),
                                  preds[user_index, :].ravel())
    
    return mean_squared_error(ratings.toarray().ravel(),
                              preds.ravel())

In [10]:
def precision_at_k(model, ratings, k=5, user_index=None):
    if not user_index:
        user_index = range(ratings.shape[0])
    ratings = ratings.tocsr()
    precisions = []
    # Note: line below may become infeasible for large datasets.
#     predictions = model.predict_for_customers()
    predictions = model.user_factors.dot(model.item_factors.T)
    for user in user_index:
        # In case of large dataset, compute predictions row-by-row like below
        # predictions = np.array([model.predict(row, i) for i in xrange(ratings.shape[1])])
        top_k = np.argsort(-predictions[user, :])[:k]
        labels = ratings.getrow(user).indices
        if np.any(labels):
            precision = float(len(set(top_k) & set(labels))) / float(k)
            precisions.append(precision)
    return np.mean(precisions) 

In [35]:
if np.any(test.getrow(0).indices):
    print('h')

In [40]:
def print_log(row, header=False, spacing=12):
    top = ''
    middle = ''
    bottom = ''
    for r in row:
        top += '+{}'.format('-'*spacing)
        if isinstance(r, str):
            middle += '| {0:^{1}} '.format(r, spacing-2)
        elif isinstance(r, int):
            middle += '| {0:^{1}} '.format(r, spacing-2)
        elif isinstance(r, float):
            middle += '| {0:^{1}.5f} '.format(r, spacing-2)
        bottom += '+{}'.format('='*spacing)
    top += '+'
    middle += '|'
    bottom += '+'
    if header:
        print(top)
        print(middle)
        print(bottom)
    else:
        print(middle)
        print(top)

In [41]:

row2 = [1,2,3,4]
row3 = [1.32,1.22,3.231,4.211]
row = ['hello','my','name','is']
# print_log(row)
print_log(row, header=True, spacing=12)
print_log(row2, header=False,spacing=12)
print_log(row3)

+------------+------------+------------+------------+
|   hello    |     my     |    name    |     is     |
+============+============+============+============+
|     1      |     2      |     3      |     4      |
+------------+------------+------------+------------+
|  1.32000   |  1.22000   |  3.23100   |  4.21100   |
+------------+------------+------------+------------+


In [75]:
def learning_curve(train, test, epochs, num_factor=50, regularization =0 , k=5, user_index=None):
    if not user_index:
        user_index = range(train.shape[0])
    prev_epoch = 0
    train_precision = []    
    test_precision = []
    topK = []
    
    train = train.astype('float64')
    
    headers = ['epochs', 'p@k train', 'p@k test','topK']
    print_log(headers, header=True)
    
    for epoch in epochs:
        model = implicit.als.AlternatingLeastSquares(factors=num_factor,iterations=epoch,regularization=regularization)
        model.fit(train.T)

        train_precision.append(precision_at_k(model, train, k, user_index))
        test_precision.append(precision_at_k(model, test, k, user_index))
        topK.append(k)
        
        row = [epoch, train_precision[-1], test_precision[-1],k]
        
        print_log(row)
        prev_epoch = epoch
    return model, train_precision, test_precision, topK

In [40]:
learning_curve(train,test,epochs=[5,10,15,20],k=10)

+------------+------------+------------+------------+
|   epochs   | p@k train  |  p@k test  |    topK    |
+============+============+============+============+
|     5      |  0.26465   |  0.00975   |     10     |
+------------+------------+------------+------------+
|     10     |  0.26048   |  0.01046   |     10     |
+------------+------------+------------+------------+
|     15     |  0.25942   |  0.01024   |     10     |
+------------+------------+------------+------------+
|     20     |  0.25878   |  0.01024   |     10     |
+------------+------------+------------+------------+


(<implicit.als.AlternatingLeastSquares at 0x5476d07ba8>,
 [0.26464600425402612,
  0.26048313582497723,
  0.25942469360883219,
  0.25877646105540369],
 [0.0097542437294147466,
  0.010463643273372181,
  0.010235621991385864,
  0.010235621991385864],
 [10, 10, 10, 10])

In [65]:
import copy

In [76]:
def grid_search_learning_curve(train, test, param_grid,
                               user_index=None, patk=5, epochs=range(2, 40, 2)):
    """
    "Inspired" (stolen) from sklearn gridsearch
    https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/model_selection/_search.py
    """
    curves = []
    keys, values = zip(*param_grid.items())
    for v in itertools.product(*values):
        params = dict(zip(keys, v))
        
        print_line = []
        for k, v in params.items():            
            print_line.append((k, v))

        print(' | '.join('{}: {}'.format(k, v) for (k, v) in print_line))
        _, train_patk, test_patk, topK = learning_curve(train, test, epochs, params.get('factors'), params.get('regularization'),
                                                        patk, user_index=user_index)
        curves.append({'params': params,
                       'patk': {'train': train_patk, 'test': test_patk},
                       })
    return curves

In [82]:
param_grid = {'factors': [5,10,20,30,40,50],
              'regularization': [0.0, 1e-5, 1e-3, 1e-1,1e0, 1e1, 1e2]}

In [83]:
%time eva = grid_search_learning_curve(train,test,param_grid,patk=10,epochs = range(10,41,10))

factors: 5 | regularization: 0.0
+------------+------------+------------+------------+
|   epochs   | p@k train  |  p@k test  |    topK    |
+============+============+============+============+
|     10     |  0.12819   |  0.02148   |     10     |
+------------+------------+------------+------------+
|     20     |  0.13201   |  0.02214   |     10     |
+------------+------------+------------+------------+
|     30     |  0.13000   |  0.02186   |     10     |
+------------+------------+------------+------------+
|     40     |  0.13068   |  0.02194   |     10     |
+------------+------------+------------+------------+
factors: 5 | regularization: 1e-05
+------------+------------+------------+------------+
|   epochs   | p@k train  |  p@k test  |    topK    |
+============+============+============+============+
|     10     |  0.13021   |  0.02181   |     10     |
+------------+------------+------------+------------+
|     20     |  0.13210   |  0.02247   |     10     |
+------------+

## Learning to Rank
- lightFM

In [16]:
from lightfm import LightFM
import lightfm
from lightfm.evaluation import precision_at_k,recall_at_k,reciprocal_rank

In [58]:
def learning_curve_lightfm(model, train, test, eval_train,
                        iterarray, user_features=None,
                        item_features=None, k=5,
                        **fit_params):
    """calculate learning curve for a lightfm reccommender
    
    parameters
    ----------
    
    model: LightFM model    
            
    train: csr matrix
        training set (indcluding all users)        
    eval_train: csr matrix
        evaluate set (remove all non-evaluate<not in test> users' data) 
    test: csr matrix 
        test set (provide ans in a eval_train users' sets)
    iterarray: list
        numbers of epochs to be evaluated
    k: int
        numbers of recommended items
    user_features: np.float32 csr_matrix of shape [n_users, n_user_features], optional
        Each row contains that user’s weights over features.
    item_features: np.float32 csr_matrix of shape [n_items, n_item_features], optional
        Each row contains that item’s weights over features.    
        
    return
    ------
    model: LightFM 
        reccommender model     
    train_patk: float
        precision at k in (eval_)training data 
    test_patk: float
        precision at k in testing data
    train_ratk: float
        recall at k in (eval_)training data
    test_ratk: float
        recall at k in testing data
    train_rrk: float
        reciprocal_rank at k in (eval_)training data
    test_rrk: float
        reciprocal_rank at k in test data
    """
    old_epoch = 0
    
    train_patk = [] # precision at k (train)
    test_patk = [] # precision at k (eval- train)
    train_ratk = [] # recall at k (eval train)
    test_ratk = [] # recall at k (test)
    train_rrk = [] # reciprocal rank (eval train)
    test_rrk = [] # reciprocal rank (test)
    
    
    headers = ['Epoch', 
               'train p@' + str(k), 
               'test p@' + str(k)
#                'train r@' + str(k),
#                'test r@' + str(k),
#                'train rr@' + str(k),
#                'test rr@' + str(k)
               ]
    
    print_log(headers, header=True)
    
    for epoch in iterarray:
        more = epoch - old_epoch
        model.fit_partial(train, 
                          user_features=user_features,
                          item_features=item_features,
                          epochs=more, **fit_params)
        ## precision at k 
        this_test_pk = precision_at_k(model, test, train_interactions=train, k=k)
        this_train_pk = precision_at_k(model, eval_train, train_interactions=train, k=k)
        train_patk.append(np.mean(this_train_pk)) # store into list
        test_patk.append(np.mean(this_test_pk))
        
        ## recall at k 
#         this_test_rk = recall_at_k(model,test,train_interactions=train,k=k)
#         this_train_rk = recall_at_k(model,eval_train,train_interactions=train,k=k)
#         train_ratk.append(np.mean(this_train_rk))
#         test_ratk.append(np.mean(this_test_rk))
                
        ## reciprocal_rank at k
#         this_test_rrk = reciprocal_rank(model,test)
#         this_train_rrk = reciprocal_rank(model,eval_train)
        
#         train_rrk.append(np.mean(this_train_rrk))
#         test_rrk.append(np.mean(this_test_rrk))
        ## print log 
        row = [epoch, float(train_patk[-1]),float(test_patk[-1])] 
        print_log(row)
        
        old_epoch = epoch
        
    return model, train_patk, test_patk

In [62]:
eval_train = train.copy()    
non_eval_users = list(set(range(train.shape[0])) - set(user_idxs)) ## 
eval_train = eval_train.tolil()
for u in non_eval_users:
    eval_train[u,:] = 0.0
eval_train = eval_train.tocsr()

model = LightFM(learning_rate=0.01, loss='warp')

model_k10, train_p10, test_p10 = learning_curve_lightfm(model,train,test,eval_train,iterarray=[1,10,50,100,150,200],k=10)

+------------+------------+------------+
|   Epoch    | train p@10 | test p@10  |
+============+============+============+
|     1      |  0.07574   |  0.01957   |
+------------+------------+------------+
|     10     |  0.08129   |  0.02013   |
+------------+------------+------------+
|     50     |  0.10727   |  0.02342   |
+------------+------------+------------+
|    100     |  0.12274   |  0.02431   |
+------------+------------+------------+
|    150     |  0.13712   |  0.02482   |
+------------+------------+------------+
|    200     |  0.14710   |  0.02472   |
+------------+------------+------------+


In [67]:
import itertools
import copy

In [72]:
#%% grid search 
def grid_search_learning_curve_lightfm(base_model, train, test, eval_train,param_grid,epochs,
                               atk=10):        
    """grid search 
    
    "Inspired" (stolen) from sklearn gridsearch
    https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/model_selection/_search.py
    
    """
    curves = []
    keys, values = zip(*param_grid.items())
    for v in itertools.product(*values):
        params = dict(zip(keys, v))
        this_model = copy.deepcopy(base_model)
        print_line = []
        for k, v in params.items():
            setattr(this_model, k, v)
            print_line.append((k, v))

        print(' | '.join('{}: {}'.format(k, v) for (k, v) in print_line))
        _, train_patk, test_patk = learning_curve_lightfm(this_model, train, test,eval_train,epochs, k=atk)
        
        curves.append({'params': params,
                       'patk': {'train': train_patk, 'test': test_patk}
#                        'ratk': {'train': train_ratk, 'test': test_ratk},
#                        'rratk':{'train': train_rratk, 'test':test_rratk}
                       })
    return curves

In [73]:
grid = {
            'loss':['bpr','warp'],
            'learning_rate':[1,0.1,0.05,0.01]
        }
        
curves = grid_search_learning_curve_lightfm(model,
                                    train,
                                    test,
                                    eval_train,
                                    grid,
                                    epochs=[1,10,50,100,150,200],
                                    atk=10)

loss: bpr | learning_rate: 1
+------------+------------+------------+
|   Epoch    | train p@10 | test p@10  |
+============+============+============+
|     1      |  0.15769   |  0.02408   |
+------------+------------+------------+
|     10     |  0.16665   |  0.01836   |
+------------+------------+------------+
|     50     |  0.17979   |  0.01393   |
+------------+------------+------------+
|    100     |  0.18060   |  0.01256   |
+------------+------------+------------+
|    150     |  0.17784   |  0.01185   |
+------------+------------+------------+
|    200     |  0.17589   |  0.01119   |
+------------+------------+------------+
loss: bpr | learning_rate: 0.1
+------------+------------+------------+
|   Epoch    | train p@10 | test p@10  |
+============+============+============+
|     1      |  0.14862   |  0.02494   |
+------------+------------+------------+
|     10     |  0.15824   |  0.02421   |
+------------+------------+------------+
|     50     |  0.16551   |  0.02051  